<a href="https://colab.research.google.com/github/felixsimard/comp551-p3/blob/main/P3_G19_COMP551.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **COMP 551 - Applied Machine Learning**

### MiniProject 3: Multi-Label Classification of Image Data

Hassan Gaber () <br>
Anthony Porporino () <br>
Felix Simard (260865674)

**Group 19** <br>
Nov 25th, 2021